<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/transformers_speecht5_tts_repro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece datasets soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00


## Speech T5 TTS Processor + Generate Works!

In [2]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

inputs = processor(text="Hello, my dog is cute", return_tensors="pt")

# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)


Generating validation split: 0 examples [00:00, ? examples/s]

In [3]:
from IPython.display import Audio

sampling_rate = 16000
Audio(speech.numpy(), rate=sampling_rate)

In [7]:
speech

tensor([-7.4343e-06,  2.7404e-04,  4.0242e-04,  ...,  1.4133e-04,
         3.6934e-05, -3.6820e-04])

## Pipeline is broken for SpeechT5 TTS

In [4]:
from transformers import pipeline

In [5]:
speecht5_tts = pipeline("text-to-audio", "microsoft/speecht5_tts")

In [8]:
speech = speecht5_tts("Hello, my dog is cute!")

In [9]:
speech

{'audio': array([8.48884447e-05, 3.29771952e-04, 1.12251044e-04, ...,
        1.24346919e-03, 1.58513710e-03, 2.44618114e-03], dtype=float32),
 'sampling_rate': 16000}

In [10]:
from IPython.display import Audio

sampling_rate = 16000
Audio(speech["audio"], rate=sampling_rate)